# pip install

In [ ]:
# !pip install --upgrade pymupdf
# !pip install PyPDF2
# !pip install pdfplumber
# !pip install langchain
# !pip install --upgrade transformers
# !pip install --upgrade pillow
# !pip install sentence_transformers
# !pip install faiss-cpu

# extract pdf_text

In [1]:
import fitz
import requests
import io
import pandas as pd
import os
import PyPDF2
import pdfplumber

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [ ]:
file_path = r'/content/gdrive/MyDrive/MBA/Data Science Practicum 23/final_project/PDF files.txt'
file = open(file_path, 'r')

# Read the contents of the file
pdfs = file.read()

lines_pdfs = pdfs.split('\n')

In [ ]:
import aiohttp
import asyncio
import fitz
import pandas as pd

num_of_doc = 500
concurrent_tasks = 10  # Limiting to 10 concurrent tasks

async def worker(session, url, semaphore):
    async with semaphore:
        url = url.replace("\r", "")
        async with session.get(url) as response:
            file_data = await response.read()

        try:
            doc = fitz.open(stream=file_data, filetype='pdf')
        except Exception as e:
            print(f"Failed to open PDF from {url}. Error: {e}")
            return None

        text = " ".join([page.get_text() for page in doc])
        total_text_len = len(text)

        return {
            "pdf_name": url.split('/')[-1],
            "url": url,
            "page_count": doc.page_count,
            "text_len": total_text_len,
            "table_of_content": doc.get_toc(),
            "pdf_text": text
        }

async def fetch_pdfs(urls):
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(concurrent_tasks)
        tasks = [worker(session, url, semaphore) for url in urls[:num_of_doc]]
        return await asyncio.gather(*tasks)

%autoawait True

# Assuming lines_pdfs is a list of URLs
data = await fetch_pdfs(lines_pdfs)
df = pd.DataFrame([d for d in data if d is not None])  # Filtering out None values


In [ ]:
# df.to_csv("df_500_url.csv", index=False, escapechar='\\')

In [3]:
df = pd.read_csv(r"/Users/yehocoh/Projects/Python/esg_project/df_500_url.csv")

In [4]:
df.sample(5)

,pdf_name,url,page_count,text_len,table_of_content,pdf_text
237,NYSE_ALB_2014.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,166,248186,"[[1, '2015 GRI Cover Page_V1', 1], [1, 'GRI an...",TABLE OF CONTENTS\nTable of Contents\n1.1 Sta...
295,Allegro-MicroSystems-Climate-Change-Report.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,2,2619,[],ACTIVITY GROUP PERFORMANCE\nElectrical & elect...
131,corporate-responsibility-2015.PDF,https://sust-reports-prod-v1.s3.us-east-2.amaz...,32,68763,"[[1, 'Message from our President and CEO', 3],...",FY2015\nCorporate \nResponsibility \nReport\n...
159,annual-report-esg-pages_2022.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,12,38495,[],"Our handprint, footprint and culture\nESG\nWe ..."
283,NASDAQ_ALKS_2021.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,43,120955,"[[1, 'SECTION 1 ', 3], [2, 'About \\nThis Repo...",1\nOctober 2021\nCorporate \nResponsibility ...
